In [37]:
from os import environ
from datetime import datetime, timedelta

from dotenv import load_dotenv
from psycopg2 import extensions, connect
import pandas as pd
from xhtml2pdf import pisa

YESTERDAY_DATE = datetime.strftime(datetime.now() - timedelta(1), '%d-%m-%Y')


def get_db_connection() -> extensions.connection:
    """Returns a connection to the AWS Bandcamp database"""

    try:
        return connect(user=environ["DB_USER"],
                       password=environ["DB_PASSWORD"],
                       host=environ["DB_IP"],
                       port=environ["DB_PORT"],
                       database=environ["DB_NAME"])
    except ConnectionError:
        print("Error: Cannot connect to the database")

In [38]:
def load_all_data(db_connection: extensions.connection) -> pd.DataFrame:
    """Loads all the data from the database into a pandas dataframe"""

    with db_connection.cursor() as curr:

        curr.execute("""
                    SELECT sale_event.*, country.country, artist.artist_name, genre.genre, item_type.item_type, item.item_name
                    FROM sale_event
                    JOIN country
                    ON country.country_id = sale_event.country_id
                    JOIN item
                    ON item.item_id = sale_event.item_id
                    JOIN artist
                    ON artist.artist_id = item.artist_id
                    JOIN item_genre
                    ON item_genre.item_id = item.item_id
                    JOIN genre
                    ON genre.genre_id =item_genre.genre_id
                    JOIN item_type
                    ON item_type.item_type_id = item.item_type_id;""")
        tuples = curr.fetchall()
        column_names = ['sale_id', 'sale_time', 'amount', 'item_id',
                        'country_id', 'country', 'artist', 'genre', 'item_type', 'item_name']

        df = pd.DataFrame(tuples, columns=column_names)

        return df

In [39]:
load_dotenv()
connection = get_db_connection()
data = load_all_data(connection)
data

,sale_id,sale_time,amount,item_id,country_id,country,artist,genre,item_type,item_name
0,127,2024-01-08 12:16:57+00:00,127,4,2,United Kingdom,the ambient drones of bill baxter,soundscapes,album,Procyon
1,51,2024-01-08 12:12:34+00:00,381,4,2,United Kingdom,the ambient drones of bill baxter,soundscapes,album,Procyon
2,2,2024-01-08 12:10:11+00:00,127,4,2,United Kingdom,the ambient drones of bill baxter,soundscapes,album,Procyon
3,127,2024-01-08 12:16:57+00:00,127,4,2,United Kingdom,the ambient drones of bill baxter,lowercase,album,Procyon
4,51,2024-01-08 12:12:34+00:00,381,4,2,United Kingdom,the ambient drones of bill baxter,lowercase,album,Procyon
...,...,...,...,...,...,...,...,...,...,...
1610,309,2024-01-08 12:40:52+00:00,254,585,1,Australia,switzerland,indie,album,Switzology
1611,309,2024-01-08 12:40:52+00:00,254,585,1,Australia,switzerland,pop,album,Switzology
1612,309,2024-01-08 12:40:52+00:00,254,585,1,Australia,switzerland,rock,album,Switzology
1613,309,2024-01-08 12:40:52+00:00,254,585,1,Australia,switzerland,twinkly,album,Switzology


In [40]:
unique_sales = data.drop_duplicates(subset='sale_id', keep='first')
album_sales = unique_sales.drop(unique_sales[unique_sales['artist'] != 'we rob rave'].index)
album_sales

,sale_id,sale_time,amount,item_id,country_id,country,artist,genre,item_type,item_name


In [41]:
unique_genre_count = data['genre'].value_counts().head(5).reset_index()
unique_genre_count

,genre,count
0,electronics,161
1,techno,70
2,house,43
3,experimental,31
4,rock,30


In [42]:
uunique_sales = data.drop_duplicates(subset='sale_id', keep='first')
album_sales = unique_sales.drop(unique_sales[unique_sales['item_type'] == 'track'].index)
popular_albums = album_sales['item_name'].value_counts().sort_values(ascending=False).head(
        5).reset_index()
popular_albums


,item_name,count
0,Timber Creek EP,4
1,Procyon,3
2,LIVE IN PARIS 1973,3
3,How To Disappear,2
4,Motorbike,2


In [43]:
unique_sales = data.drop_duplicates(subset='sale_id', keep='first')
album_sales = unique_sales.drop(
     unique_sales[unique_sales['item_type'] == 'track'].index)
album_sales = album_sales.groupby(
    'item_name')['amount'].sum()

album_sales = (
     album_sales/100).sort_values(ascending=False).head(5).reset_index()
album_sales


,item_name,amount
0,"After Fifteen Years, Live at Buddy",34.58
1,"Orphans: Brawlers, Bawlers & Bastards (Remaste...",30.00
2,LIVE IN PARIS 1973,29.97
3,Timber Creek EP,26.76
4,Dark Current,25.00


In [59]:
country_sales = data['country'].value_counts(
).sort_values(ascending=False).reset_index()
country_sales
most_popular_artists = data.groupby('country')['artist'].apply(
    lambda x: x.value_counts().idxmax()).reset_index()
most_popular_artists
final = pd.merge(country_sales, most_popular_artists).to_dict('records')
final

[{'country': 'Germany', 'count': 325, 'artist': 'dj heartstring'},
 {'country': 'United Kingdom',
  'count': 233,
  'artist': 'the ambient drones of bill baxter'},
 {'country': 'United States', 'count': 224, 'artist': 'the von tramps'},
 {'country': 'Australia', 'count': 143, 'artist': 'keith renshaw'},
 {'country': 'Japan', 'count': 103, 'artist': 'resin tomb'},
 {'country': 'Republic of Korea', 'count': 103, 'artist': 'doctor jeep'},
 {'country': 'France', 'count': 86, 'artist': 'lunatic alien records'},
 {'country': 'Netherlands', 'count': 55, 'artist': 'agonist999'},
 {'country': 'Spain', 'count': 49, 'artist': 'emilio dahmen'},
 {'country': 'Hong Kong', 'count': 40, 'artist': 'othercide records'},
 {'country': 'Switzerland', 'count': 36, 'artist': 'red axes'},
 {'country': 'Canada', 'count': 28, 'artist': 'walden'},
 {'country': 'Finland', 'count': 27, 'artist': 'juul exler'},
 {'country': 'Singapore', 'count': 22, 'artist': 'soulely'},
 {'country': 'Italy', 'count': 22, 'artist':

In [62]:
unique_sales = data.drop_duplicates(subset='sale_id', keep='first')

track_sales = unique_sales.drop(
    unique_sales[unique_sales['item_type'] == 'album'].index)
popular_tracks = track_sales['item_name'].value_counts().sort_values(ascending=False).head(
    5).reset_index()
selected = popular_tracks['item_name'].to_list()

track_sales = data[data['item_type'] == 'track']

filtered_track_sales = track_sales[track_sales['item_name'].isin(selected)]

track_genre = filtered_track_sales.groupby(['item_name'])[
    'genre'].agg(list).reset_index()

track_genre['genre'] = track_genre['genre'].apply(
    lambda x: list(set(x)))

final = pd.merge(popular_tracks, track_genre).to_dict('records')
track_sales

,sale_id,sale_time,amount,item_id,country_id,country,artist,genre,item_type,item_name
36,1,2024-01-08 12:10:10+00:00,254,70,16,Netherlands,anunaku & dj plead,other,track,Wheele
37,3,2024-01-08 12:10:22+00:00,100,71,3,Germany,lon,electronics,track,Marcos Valle - Estrelar (LON Remix)
38,3,2024-01-08 12:10:22+00:00,100,71,3,Germany,lon,brazilian music,track,Marcos Valle - Estrelar (LON Remix)
39,3,2024-01-08 12:10:22+00:00,100,71,3,Germany,lon,indie,track,Marcos Valle - Estrelar (LON Remix)
40,3,2024-01-08 12:10:22+00:00,100,71,3,Germany,lon,disco,track,Marcos Valle - Estrelar (LON Remix)
...,...,...,...,...,...,...,...,...,...,...
1599,307,2024-01-08 12:40:33+00:00,125,583,3,Germany,abderraouf b grissa & dan drohan,african,track,Visco City
1600,307,2024-01-08 12:40:33+00:00,125,583,3,Germany,abderraouf b grissa & dan drohan,funk,track,Visco City
1601,307,2024-01-08 12:40:33+00:00,125,583,3,Germany,abderraouf b grissa & dan drohan,latin,track,Visco City
1602,307,2024-01-08 12:40:33+00:00,125,583,3,Germany,abderraouf b grissa & dan drohan,jazz,track,Visco City


,country,artist
0,Australia,keith renshaw
1,Austria,jungle
2,Belgium,oerheks
3,Brazil,gustavo jobim
4,Bulgaria,ulterior motive
5,Canada,walden
6,China,not yes
7,Denmark,coido
8,Finland,juul exler
9,France,lunatic alien records


In [47]:
unique_sales = data.drop_duplicates(subset='sale_id', keep='first')
album_sales = unique_sales.drop(
    unique_sales[unique_sales['item_type'] == 'track'].index)
popular_albums = album_sales['item_name'].value_counts().sort_values(ascending=False).head(
    5).reset_index()
selected = popular_albums['item_name'].to_list()

album_sales = data[data['item_type'] == 'album']

filtered_album_sales = album_sales[album_sales['item_name'].isin(selected)]

albums_genre = filtered_album_sales.groupby(['item_name'])[
'genre'].agg(list).reset_index()

albums_genre['genre'] = albums_genre['genre'].apply(
lambda x: list(set(x)))

pd.merge(popular_albums, albums_genre).to_dict('records')



[{'item_name': 'Timber Creek EP',
  'count': 4,
  'genre': ['rock',
   'alternative',
   'indi blues roots',
   'country',
   'indi pop comedy',
   'fremantle']},
 {'item_name': 'Procyon',
  'count': 3,
  'genre': ['creative commons',
   'experimental',
   'space ambient',
   'dark ambient',
   'chill out',
   'drone',
   'ambient',
   'lowercase',
   'ambient drone',
   'soundscapes',
   'atmospheric',
   'longform']},
 {'item_name': 'LIVE IN PARIS 1973',
  'count': 3,
  'genre': ['rock', 'german rock', 'experimental', 'krautrock']},
 {'item_name': 'How To Disappear',
  'count': 2,
  'genre': ['techno',
   'ambient',
   'meditative',
   'drone ambient',
   'meditation music',
   'new age']},
 {'item_name': 'Motorbike',
  'count': 2,
  'genre': ['punk', 'power pop', 'post punk']}]